In [1]:
import tensorflow as tf
import ktrain

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

Default GPU Device:/device:GPU:0


In [2]:

import ktrain
from ktrain import text

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

num_folds = 5

run_precision = []
run_recall = []
run_f1score = []
run_accuracy = []

kf = KFold(n_splits=5, shuffle=False, random_state=125)

In [4]:
data = pd.read_csv('sstubsIn_edited.csv')

In [5]:
print(data.shape)

(10231, 5)


In [6]:
#print("pre-processing train data...")

from sklearn.utils import shuffle
from tqdm import tqdm

#label_names = ["bugType"]
#y_train = data[label_names].values

raw_docs_train = data['sourceBeforeFix'].tolist()
#raw_docs_test = test_data['message'].tolist() 
num_classes = 4

#processed_docs_train = []
#for doc in tqdm(raw_docs_train):
#    tokens = word_tokenize(doc)
#    filtered = [word for word in tokens if word not in stop_words]
#    processed_docs_train.append(" ".join(filtered))
#end for
'''
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for
'''


#print("tokenizing input data...")
#tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
#tokenizer.fit_on_texts(processed_docs_train )  #leaky
#word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
#word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
#word_index = tokenizer.word_index
#print("dictionary size: ", len(word_index))

#pad sequences
#word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train = raw_docs_train
word_seq_train = np.array(word_seq_train)

#data.bugType.value_counts()

In [7]:
class_names = [0,1,2,3]

In [8]:
encoding = {
    'CHANGE_IDENTIFIER': 0, 
    'CHANGE_MODIFIER': 0,
    'CHANGE_NUMERAL': 0, 
    'SWAP_BOOLEAN_LITERAL': 0,
     'DIFFERENT_METHOD_SAME_ARGS': 1, 
     'OVERLOAD_METHOD_MORE_ARGS': 1, 
    'OVERLOAD_METHOD_DELETED_ARGS': 1,
     'CHANGE_CALLER_IN_FUNCTION_CALL': 1, 
     'SWAP_ARGUMENTS': 1,
     'CHANGE_OPERATOR': 2, 
    'CHANGE_UNARY_OPERATOR': 2, 
    'CHANGE_OPERAND': 2, 
     'LESS_SPECIFIC_IF': 3,
    'MORE_SPECIFIC_IF': 3,
     'ADD_THROWS_EXCEPTION': 3, 
      'DELETE_THROWS_EXCEPTION':3,
       
}

In [9]:
y_train_initial= data['bugType'].values


y_train_initial = [encoding[x] for x in y_train_initial]
y_train_initial=np.array(y_train_initial)

In [10]:
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
count = 1

for train_index, test_index in kf.split(word_seq_train):
    x_trn, x_tst = word_seq_train[train_index], word_seq_train[test_index]
    y_trn, y_tst = y_train_initial[train_index], y_train_initial[test_index]
    
    x_new_train, x_val, y_new_train, y_val= train_test_split(x_trn, y_trn, test_size=0.11115, random_state=125)
    
    print("\nFold ", count)
    
    (x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_new_train, y_train=y_new_train,
                                                                      x_test=x_tst, y_test=y_tst,
                                                                       class_names=class_names,
                                                                       preprocess_mode='standard',
                                                                       maxlen=150, 
                                                                       max_features=1000)
    
    model = text.text_classifier('bigru', train_data=(x_train, y_train), preproc=preproc)
    
    learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=10)
    
    
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    #learner.lr_find()

    
    #learner.fit_onecycle(2e-5, 40, callbacks=[callbacks])

    learner.autofit(0.01)

    
    #predictor = ktrain.get_predictor(learner.model, preproc)
    #predictor.get_classes()
    
    (x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_val, y_train=y_val,
                                                                       x_test=x_val, y_test=y_val,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='standard',
                                                                       maxlen=150, 
                                                                       max_features=1000)
    
    learner.validate(val_data=(x_test, y_test), class_names=class_names)
    
    '''
    print(metrics.classification_report(y_tst, y_pred))
    
    precision = precision_score(y_tst, y_pred, pos_label=1)
    recall = recall_score(y_tst, y_pred, pos_label=1)
    f1score = f1_score(y_tst, y_pred, pos_label=1)
    lstm_accuracy = accuracy_score(y_tst, y_pred)

    run_accuracy.append(accuracy)
    run_f1score.append(f1score)
    run_precision.append(precision)
    run_recall.append(recall)
    '''
    count = count+1
    #X_train = data['sourceBeforeFix'].tolist()


Fold  1
language: en
Word Counts: 7750
Nrows: 7274
7274 train sequences
train sequence lengths:
	mean : 5
	95percentile : 12
	99percentile : 45
x_train shape: (7274,150)
y_train shape: (7274, 4)
Is Multi-Label? False
2047 test sequences
test sequence lengths:
	mean : 2
	95percentile : 7
	99percentile : 15
x_test shape: (2047,150)
y_test shape: (2047, 4)
task: text classification
Is Multi-Label? False
compiling word ID features...
maxlen is 150
word vectors will be loaded from: https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
processing pretrained word vectors...
downloading pretrained word vectors to /home/amiangshu/ktrain_data ...
[██████████████████████████████████████████████████]
extracting pretrained word vectors...
done.

cleanup downloaded zip...
done.

loading pretrained word vectors...this may take a few moments...


done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 16s 19ms/step - loss: 0.8736 - accuracy: 0.6277 - val_loss: 0.7486 - val_accuracy: 0.6541
Epoch 2/1024
728/728 [==============================] - 13s 18ms/step - loss: 0.6718 - accuracy: 0.6908 - val_loss: 0.7787 - val_accuracy: 0.6546
Epoch 3/1024
728/728 [==============================] - 12s 17ms/step - loss: 0.6605 - accuracy: 0.6815 - val_loss: 0.8569 - val_accuracy: 0.6556

Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
Epoch 4/1024
728/728 [==============================] - 13s 18ms/step - loss: 0.6141 - accuracy: 0.7084 - val_loss: 0.8431 - val_accuracy: 0.6571
Epoch 5/1024
728/728 [==============================] - 12s 17ms/step - loss: 0.5682 - accuracy: 0.7167 - val_loss: 0.8390 - val_accura

done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 16s 20ms/step - loss: 0.8903 - accuracy: 0.6108 - val_loss: 0.6706 - val_accuracy: 0.7048
Epoch 2/1024
728/728 [==============================] - 15s 21ms/step - loss: 0.7088 - accuracy: 0.6776 - val_loss: 0.6843 - val_accuracy: 0.7058
Epoch 3/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.7183 - accuracy: 0.6745 - val_loss: 0.7361 - val_accuracy: 0.6794

Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
Epoch 4/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.6183 - accuracy: 0.7025 - val_loss: 0.7066 - val_accuracy: 0.6960
Epoch 5/1024
728/728 [==============================] - 15s 21ms/step - loss: 0.6041 - accuracy: 0.7024 - val_loss: 0.7440 - val_accura

done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 31s 40ms/step - loss: 0.8625 - accuracy: 0.6255 - val_loss: 0.8575 - val_accuracy: 0.6388
Epoch 2/1024
728/728 [==============================] - 29s 39ms/step - loss: 0.6490 - accuracy: 0.7005 - val_loss: 0.8620 - val_accuracy: 0.6256
Epoch 3/1024
728/728 [==============================] - 32s 44ms/step - loss: 0.6099 - accuracy: 0.7133 - val_loss: 0.8608 - val_accuracy: 0.6300

Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
Epoch 4/1024
728/728 [==============================] - 31s 43ms/step - loss: 0.5749 - accuracy: 0.7300 - val_loss: 0.9592 - val_accuracy: 0.6227
Epoch 5/1024
728/728 [==============================] - 29s 40ms/step - loss: 0.5516 - accuracy: 0.7289 - val_loss: 0.9775 - val_accura

done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 16s 19ms/step - loss: 0.8505 - accuracy: 0.6307 - val_loss: 0.8726 - val_accuracy: 0.6197
Epoch 2/1024
728/728 [==============================] - 15s 20ms/step - loss: 0.6875 - accuracy: 0.6901 - val_loss: 0.9260 - val_accuracy: 0.5919
Epoch 3/1024
728/728 [==============================] - 14s 20ms/step - loss: 0.6999 - accuracy: 0.6906 - val_loss: 0.9542 - val_accuracy: 0.5909

Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
Epoch 4/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.5899 - accuracy: 0.7203 - val_loss: 0.9870 - val_accuracy: 0.5992
Epoch 5/1024
728/728 [==============================] - 15s 21ms/step - loss: 0.5813 - accuracy: 0.7158 - val_loss: 0.9922 - val_accura

/home/amiangshu/anaconda3/envs/dsaktrain/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.67      0.90      0.77       587
           1       0.36      0.16      0.23       237
           2       0.20      0.08      0.12        36
           3       0.00      0.00      0.00        50

    accuracy                           0.63       910
   macro avg       0.31      0.29      0.28       910
weighted avg       0.54      0.63      0.56       910


Fold  5
language: en
Word Counts: 7506
Nrows: 7275
7275 train sequences
train sequence lengths:
	mean : 5
	95percentile : 11
	99percentile : 47
x_train shape: (7275,150)
y_train shape: (7275, 4)
Is Multi-Label? False
2046 test sequences
test sequence lengths:
	mean : 3
	95percentile : 8
	99percentile : 17
x_test shape: (2046,150)
y_test shape: (2046, 4)
task: text classification
Is Multi-Label? False
compiling word ID features...
maxlen is 150
word vectors will be loaded from: https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
processing 

done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 16s 20ms/step - loss: 0.8563 - accuracy: 0.6384 - val_loss: 0.8135 - val_accuracy: 0.6124
Epoch 2/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.6747 - accuracy: 0.6841 - val_loss: 0.8248 - val_accuracy: 0.6056
Epoch 3/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.6362 - accuracy: 0.7081 - val_loss: 0.8638 - val_accuracy: 0.6134

Epoch 00003: Reducing Max LR on Plateau: new max lr will be 0.005 (if not early_stopping).
Epoch 4/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.6167 - accuracy: 0.7059 - val_loss: 0.8554 - val_accuracy: 0.6080
Epoch 5/1024
728/728 [==============================] - 14s 19ms/step - loss: 0.5607 - accuracy: 0.7345 - val_loss: 0.8746 - val_accura